In [97]:
#Accessing API through SDK
from IPython.display import clear_output #Adding interactive control
import ipywidgets as wdg #Adding interactive control
import pandas as pd
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API #Import API

#Filtering the data for England only
filters = ['areaType=nation','areaName=England']

# Values for the graphs
structure = {"cases": "newCasesByPublishDate","tests": "newTestsByPublishDate","date": "date"}

#Accessing API
api = Cov19API(filters=filters, structure=structure)
timeseries=api.get_json()

#Saving API queries in json file
with open("timeseries.json", "wt") as OUTF:
    json.dump(timeseries, OUTF)


#Data Visualisation

# enables the embedding of matplotlib output
%matplotlib inline
# to make figures larger
plt.rcParams['figure.dpi'] = 100

#Loading data from JSON file
with open("timeseries.json", "rt") as INFILE:
    data=json.load(INFILE)

#Wrangling the data
#to make x-axis to be date and sort them
datalist=data['data']
dates=[dict['date'] for dict in datalist ]
dates.sort()

#Converting date string to pandas datetime object
def parse_date(datestr):
    return pd.to_datetime(datestr, format="%Y-%m-%d")

#to show the start date and end date
start_date=parse_date(dates[0])
end_date=parse_date(dates[-1])
print ("This graph shows a comparison of the number of new cases with the number of new tests.")
print (startdate, ' to ', enddate)

#Creating date index
index=pd.date_range(start_date, end_date, freq='D')
casesdf=pd.DataFrame(index=index, columns=['cases', 'tests'])

#Filling values into the DataFrame
for entry in datalist:
    date=parse_date(entry['date'])
    for column in ['cases', 'tests']: #for y-axis
        if pd.isna(casesdf.loc[date, column]): 
            value= float(entry[column]) if entry[column]!=None else 0.0
            casesdf.loc[date, column]=value # Graph plotted
            
# to fill in any remaining missing data
casesdf.fillna(0.0, inplace=True)            

#Saving data into pickle file
casesdf.to_pickle("timeseriesdf.pkl")

#Adding Interactive Controls
#Updating the data
#Adding clickable buttons
    
def access_api(button):
    
    #Code for polling API:
    filters = ['areaType=nation','areaName=England']
    structure = {"cases": "newCasesByPublishDate","tests": "newTestsByPublishDate","date": "date"}
    api = Cov19API(filters=filters, structure=structure)
    timeseries=api.get_json()
    
    # Saving the fetched data as json
    with open("timeseries.json", "wt") as OUTF:
        json.dump(timeseries, OUTF)
        
    # Updating the button when its refreshed, the icon is changed to the check and its disabled after completion of the refresh
    button.icon="check"
    button.disabled=True
    
# Setting up the button
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='',
    tooltip='Click to download the data of a comparison of the number of new cases with the number of tests',
    icon='download')

# Adding callback function to the button
apibutton.on_click(access_api)

# Display the button
display(apibutton)


casesdf=pd.read_pickle("timeseriesdf.pkl") #bring pickle file
casecols=wdg.SelectMultiple( # makes use of Callback
    options=['cases', 'tests'],
    value=['cases', 'tests'], # setting the value of the graph
    rows=2, # rows of the selection box
    description='Select:', #description next to the selection rows
    disabled=False)

scale=wdg.RadioButtons(
    options=['linear', 'log'], #adding linear and log scales
    description='Scale:',
    disabled=False)

controls=wdg.VBox([casecols, scale]) # the boxes are listed vertically
# Make selection to choose the graph scale to be a linear or log
def case_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False # it appears as linear if user clicks linear
    else:
        logscale=True # otherwise appears as log scale
    ncols=len(gcols)
    if ncols>0: # positive value
        casesdf[list(gcols)].plot(logy=logscale) # make y-axis log scale
        plt.show() # upadte the graph
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

graph=wdg.interactive_output(case_graph, {'gcols': casecols, 'gscale': scale}) # the output graph
display(controls, graph)


This graph shows a comparison of the number of new cases with the number of new tests.
2020-01-31 00:00:00  to  2023-11-30 00:00:00


Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download the data o…

Output()